In [1]:
import numpy as np
import math
from itertools import product
from scipy.spatial import ConvexHull
import plotly.graph_objects as go
from ipywidgets import interact, FloatSlider, fixed

In [2]:
def generate_edges_full(vertices):
    """Verbindet alle Punkte, die sich in genau einer Koordinate unterscheiden."""
    edges = []
    for i in range(len(vertices)):
        for j in range(i + 1, len(vertices)):
            diff = np.abs(vertices[i] - vertices[j])
            if np.count_nonzero(diff) == 1:
                edges.append((i, j))
    return edges

def generate_edges_convex_hull(vertices):
    """Verbindet Punkte, die zusammen eine Facette auf der konvexen Hülle bilden."""
    hull = ConvexHull(vertices)
    edges = set()
    for simplex in hull.simplices:
        for i in range(len(simplex)):
            for j in range(i + 1, len(simplex)):
                edge = tuple(sorted((simplex[i], simplex[j])))
                edges.add(edge)
    return list(edges)

def rotation_matrix(i, j, theta):
    m = np.identity(4)
    c, s = np.cos(theta), np.sin(theta)
    m[i, i], m[i, j] = c, -s
    m[j, i], m[j, j] = s, c
    return m

def project_4d_to_3d(v, w_dist=3.5):
    factor = 1 / (w_dist - v[3]) if (w_dist - v[3]) != 0 else 1
    return v[:3] * factor

In [3]:
vertices_example = np.array([
    [-2, 3, 1, -3],
    [1, -2, -1, 2],
    [4, -4, 1, 2],
    [1, 2, 3, 0],
    [-3, 3, -1, -2],
    [1, 2, 3, -3],
    [-2, -1, -3, 0]
], dtype=float)

vertices_tesseract = np.array(list(product([-1, 1], repeat=4)), dtype=float)
edges_tesseract = generate_edges_full(vertices_tesseract)

In [10]:
def generate_clifford_torus(n=20, m=20):
    """
    Erzeugt Punkte auf einem Clifford-Torus in 4D.
    Liegt gleichmäßig auf der 3-Sphäre (Einheitskugeloberfläche in ℝ⁴).
    """
    points = []
    r = 1 / np.sqrt(2)
    for i in range(n):
        theta = 2 * np.pi * i / n
        for j in range(m):
            phi = 2 * np.pi * j / m
            x = r * np.cos(theta)
            y = r * np.sin(theta)
            z = r * np.cos(phi)
            w = r * np.sin(phi)
            points.append([x, y, z, w])
    return np.array(points)

def generate_edges_clifford_torus(n, m):
    edges = []
    for i in range(n):
        for j in range(m):
            idx = i * m + j
            idx_theta = ((i + 1) % n) * m + j
            idx_phi   = i * m + ((j + 1) % m)
            edges.append((idx, idx_theta))
            edges.append((idx, idx_phi))
    return edges

vertices_clifford = generate_clifford_torus(n=24, m=16)
edges_clifford = generate_edges_clifford_torus(n=24, m=16)

In [5]:
def get_frame(vertices, edges, theta_xy, theta_xw):
    rot_xy = rotation_matrix(0, 1, theta_xy)
    rot_xw = rotation_matrix(0, 3, theta_xw)
    transformed = [project_4d_to_3d(rot_xw @ rot_xy @ v) for v in vertices]
    transformed = np.array(transformed)

    lines = []
    for i, j in edges:
        x = [transformed[i][0], transformed[j][0]]
        y = [transformed[i][1], transformed[j][1]]
        z = [transformed[i][2], transformed[j][2]]
        lines.append(go.Scatter3d(
            x=x, y=y, z=z,
            mode='lines',
            line=dict(color='gray', width=3),
            hoverinfo='skip',
            showlegend=False
        ))

    points = go.Scatter3d(
        x=transformed[:, 0],
        y=transformed[:, 1],
        z=transformed[:, 2],
        mode='markers',
        marker=dict(size=4, color='blue'),
        hoverinfo='skip',
        showlegend=False
    )

    return lines + [points]

In [6]:
frames = []
N = 60
for i in range(N):
    theta = 2 * np.pi * i / N
    frame = go.Frame(data=get_frame(vertices_tesseract, edges_tesseract, theta, theta/2))
    frames.append(frame)

fig = go.Figure(
    data=get_frame(vertices_tesseract, edges_tesseract, 0, 0),
    layout=go.Layout(
        title="",
        margin=dict(l=0, r=0, t=0, b=0),
        scene=dict(
            xaxis=dict(showbackground=False, visible=False),
            yaxis=dict(showbackground=False, visible=False),
            zaxis=dict(showbackground=False, visible=False)
        ),
        updatemenus=[dict(
            type="buttons",
            showactive=False,
            buttons=[dict(label="▶", method="animate", args=[None, {"frame": {"duration": 50}, "fromcurrent": True}])]
        )]
    ),
    frames=frames
)

fig.show()

In [12]:
def draw_polytope(
    vertices, edges,
    theta_xy=0.0, theta_xz=0.0, theta_yz=0.0,
    theta_xw=0.0, theta_yw=0.0, theta_zw=0.0
):
    # Rotationsmatrizen (alle 6 möglichen 4D-Rotationen)
    rot_xy = rotation_matrix(0, 1, theta_xy)
    rot_xz = rotation_matrix(0, 2, theta_xz)
    rot_yz = rotation_matrix(1, 2, theta_yz)
    rot_xw = rotation_matrix(0, 3, theta_xw)
    rot_yw = rotation_matrix(1, 3, theta_yw)
    rot_zw = rotation_matrix(2, 3, theta_zw)

    # Kombinierte Gesamtrotation
    R = rot_zw @ rot_yw @ rot_xw @ rot_yz @ rot_xz @ rot_xy

    # Transformation und Projektion
    transformed = [project_4d_to_3d(R @ v) for v in vertices]
    transformed = np.array(transformed)

    # Kanten zeichnen
    edge_traces = []
    for i, j in edges:
        x = [transformed[i][0], transformed[j][0]]
        y = [transformed[i][1], transformed[j][1]]
        z = [transformed[i][2], transformed[j][2]]
        edge_traces.append(go.Scatter3d(
            x=x, y=y, z=z,
            mode='lines',
            line=dict(color='gray', width=2),
            hoverinfo='skip',
            showlegend=False
        ))

    # Punkte zeichnen
    point_trace = go.Scatter3d(
        x=transformed[:, 0],
        y=transformed[:, 1],
        z=transformed[:, 2],
        mode='markers',
        marker=dict(size=4, color='blue'),
        hoverinfo='skip',
        showlegend=False
    )

    # Plotly-Plot
    fig = go.Figure(data=edge_traces + [point_trace])
    fig.update_layout(
        margin=dict(l=0, r=0, t=0, b=0),
        scene=dict(
            xaxis=dict(showbackground=False, visible=False),
            yaxis=dict(showbackground=False, visible=False),
            zaxis=dict(showbackground=False, visible=False)
        )
    )
    fig.show()


In [ ]:
interact(
    draw_polytope,
    vertices=fixed(vertices_tesseract),
    edges=fixed(edges_tesseract),
    theta_xy=FloatSlider(min=0, max=2*np.pi, step=0.05, value=0, description='XY'),
    theta_xz=FloatSlider(min=0, max=2*np.pi, step=0.05, value=0, description='XZ'),
    theta_yz=FloatSlider(min=0, max=2*np.pi, step=0.05, value=0, description='YZ'),
    theta_xw=FloatSlider(min=0, max=2*np.pi, step=0.05, value=0, description='XW'),
    theta_yw=FloatSlider(min=0, max=2*np.pi, step=0.05, value=0, description='YW'),
    theta_zw=FloatSlider(min=0, max=2*np.pi, step=0.05, value=0, description='ZW')
)


interactive(children=(FloatSlider(value=0.0, description='XY', max=6.283185307179586, step=0.05), FloatSlider(…

<function __main__.draw_polytope(vertices, edges, theta_xy=0.0, theta_xz=0.0, theta_yz=0.0, theta_xw=0.0, theta_yw=0.0, theta_zw=0.0)>

In [ ]:
interact(
    draw_polytope,
    vertices=fixed(vertices_clifford),  # oder dein beliebiges Polytope
    edges=fixed(edges_clifford),
    theta_xy=FloatSlider(min=0, max=2*np.pi, step=0.05, value=0, description='XY'),
    theta_xz=FloatSlider(min=0, max=2*np.pi, step=0.05, value=0, description='XZ'),
    theta_yz=FloatSlider(min=0, max=2*np.pi, step=0.05, value=0, description='YZ'),
    theta_xw=FloatSlider(min=0, max=2*np.pi, step=0.05, value=0, description='XW'),
    theta_yw=FloatSlider(min=0, max=2*np.pi, step=0.05, value=0, description='YW'),
    theta_zw=FloatSlider(min=0, max=2*np.pi, step=0.05, value=0, description='ZW')
)

interactive(children=(FloatSlider(value=0.0, description='XY', max=6.283185307179586, step=0.05), FloatSlider(…

<function __main__.draw_polytope(vertices, edges, theta_xy=0.0, theta_xz=0.0, theta_yz=0.0, theta_xw=0.0, theta_yw=0.0, theta_zw=0.0)>